## Real Data Analysis

In [6]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

In [7]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # Get the parent directory
from src.pipeline.data_extraction import DataExtraction

# Load the data
data_extract = DataExtraction()
data = data_extract.extract_data_from_supabase()

# Print the first few rows of the data
data.head()

2025-07-24 20:53:25,341 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-07-24 20:53:25,343 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-24 20:53:25,401 INFO sqlalchemy.engine.Engine select current_schema()
2025-07-24 20:53:25,403 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-24 20:53:25,465 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-07-24 20:53:25,466 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-24 20:53:25,598 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-24 20:53:25,603 INFO sqlalchemy.engine.Engine SELECT network_data.id, network_data.time, network_data.latency_ms, network_data.jitter_ms, network_data.packet_loss, network_data.rssi_dbm, network_data.interface, network_data.download_mbps, network_data.upload_mbps, network_data.cpu_usage, network_data.ram_usage, network_data.device_count, network_data.temp 
FROM network_data
2025-07-24 20:53:25,604 INFO sqlalchemy.engine.Engine [generated in 0.00115s] {}
2025-07-24 20:5

,jitter_ms,id,rssi_dbm,download_mbps,cpu_usage,device_count,packet_loss,time,latency_ms,interface,upload_mbps,ram_usage,temp
0,0.768,996970c3-5cc2-4666-adb5-76b5ce1412d3,-19,100.63,5.0,6,0.0,2025-07-21 08:54:16.573397+00:00,17.173,nmap,38.39,78.3,47.2
1,0.717,4db743f2-4248-4a77-85e6-83da3f6883d7,-15,NaN,7.5,6,0.0,2025-07-21 12:44:09.485153+00:00,17.238,nmap,NaN,67.0,45.7
2,1.024,e1074c0f-d41a-450a-90b8-1971062e730d,-13,0.00,5.3,6,0.0,2025-07-21 12:51:04.539047+00:00,17.165,nmap,0.00,67.8,46.2
3,0.897,f26b6417-c473-4e4c-b755-97367c97622b,-16,0.00,7.1,5,0.0,2025-07-21 12:56:15.429612+00:00,17.177,nmap,0.00,65.6,45.2
4,0.973,10ba122a-beca-401e-aaf0-b69bc7a56c55,-22,0.00,30.0,4,0.0,2025-07-22 04:32:59.924587+00:00,17.423,nmap,0.00,15.4,39.4


### Data Preprocessing

In [8]:
# Drop unnecessary columns
data = data.drop(columns=["id"], axis=1)
print("Data columns has been dropped successfully.")

# Check for missing values
missing_values = data.isnull().sum()
print(f"Missing values in each columns : {missing_values}")

Data columns has been dropped successfully.
Missing values in each columns : jitter_ms        0
rssi_dbm         0
download_mbps    1
cpu_usage        0
device_count     0
packet_loss      0
time             0
latency_ms       0
interface        0
upload_mbps      1
ram_usage        0
temp             0
dtype: int64


In [12]:
# Check the download and upload columns value
print("Download Column Values")
print(data["download_mbps"].value_counts())

print("\n")

print("Upload Column Values")
print(data["upload_mbps"].value_counts())

Download Column Values
download_mbps
0.00      657
100.63      1
Name: count, dtype: int64


Upload Column Values
upload_mbps
0.00     657
38.39      1
Name: count, dtype: int64


In [13]:
# Dropped the columns with all missing values
data = data.drop(columns=["download_mbps", "upload_mbps"], axis=1)
print("Columns with all missing values have been dropped successfully.")

# Check the data types of the columns
print("Data Types of the Columns")
print(data.dtypes)

Columns with all missing values have been dropped successfully.
Data Types of the Columns
jitter_ms                   float64
rssi_dbm                      int64
cpu_usage                   float64
device_count                  int64
packet_loss                 float64
time            datetime64[ns, UTC]
latency_ms                  float64
interface                    object
ram_usage                   float64
temp                        float64
dtype: object
